In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#import sklearn
import sklearn.neural_network as nn
import sklearn.svm as svm
import sklearn.linear_model as lm
import sklearn.neighbors as neighbors
import sklearn.tree as tree
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

### 1. 데이터 불러오기 및 데이터 파악

In [2]:
df = pd.read_csv('./sklearn_practice.csv.txt')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


#### null 포함 여부 체크
- null 값은 가지고 있지 않음

In [3]:
df.isnull().sum().sum()

0

In [4]:
class_zero = df[df['Class'] == 0]
class_one = df[df['Class'] == 1]
print('# of zeros: %d, # of ones: %d, ones/zeros ratio: %f%%' 
      %(len(class_zero), len(class_one), len(class_one)/len(class_zero)))

# of zeros: 284315, # of ones: 492, ones/zeros ratio: 0.001730%


### 2. Training/Test Data Split
- class별 데이터가 불균형하므로 이를 위한 보완 방법 필요
 - 적은 수에 맞춰 under-sampling
 - 적은 수를 많은 수에 맞춰 수를 키워 분석

In [5]:
data = pd.concat([class_zero.sample(n=len(class_zero)), class_one.sample(n=len(class_one))]) # Return a random sample of items from an axis of object

In [6]:
train_data = data.iloc[100:-100] # Class 0, 1에서 각각 100개씩 빼고 나머지
test_data = pd.concat([data.iloc[:100], data.iloc[-100:]])

#### 모든 컬럼 사용

In [7]:
# x_train = train_data.loc[:, 'Time':'Amount']
# y_train = train_data.loc[:, 'Class']
# # test set
# x_test = test_data.loc[:, 'Time':'Amount']
# y_test = test_data.loc[:, 'Class']

#### 불필요한 Time, Amount 컬럼 제거

In [8]:
x_train = train_data.loc[:, 'V1':'V28']
y_train = train_data.loc[:, 'Class']
# test set
x_test = test_data.loc[:, 'V1':'V28']
y_test = test_data.loc[:, 'Class']

### 불필요 컬럼 제거

In [17]:
# x_train = train_data.loc[:, 'V1':'V4']
# x_test = test_data.loc[:, 'V1':'V4']
del x_train['V13']
del x_test['V13']
del x_train['V15']
del x_test['V15']
del x_train['V20']
del x_test['V20']

In [18]:
x_train.head()
x_train.shape

(284607, 25)

### 3. Train and Validation depending machine learning algorithms

#### 1) LogisticRegression()

In [19]:
model = lm.LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# performance
cnt = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test.iloc[i]:
        cnt += 1
print(cnt/len(y_pred))

0.795


#### 2) SVC

In [24]:
model = svm.SVC(gamma='scale')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# performance
cnt = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test.iloc[i]:
        cnt += 1
print(cnt/len(y_pred))

0.825


#### 3)KNN

In [20]:
model = neighbors.KNeighborsClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# performance
cnt = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test.iloc[i]:
        cnt += 1
print(cnt/len(y_pred))

0.89


#### 4) Neural network

In [23]:
model = nn.MLPClassifier(learning_rate_init=0.01)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# performance
cnt = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test.iloc[i]:
        cnt += 1
print(cnt/len(y_pred))

0.905


#### 5) Decision Tree

In [21]:
model = tree.DecisionTreeClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# performance
cnt = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test.iloc[i]:
        cnt += 1
print(cnt/len(y_pred))

0.845


#### 6) Bayesian Classification

In [22]:
from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# performance
cnt = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test.iloc[i]:
        cnt += 1
print(cnt/len(y_pred))

0.795
